In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

In [3]:
!ls data/

census_income_learn.csv census_income_test.csv


In [4]:
!ls sup_info/

census_income_additional_info.pdf data_cols.csv
census_income_metadata.txt        data_cols.numbers


In [11]:
def append_anomaly_counts(df_info=None, dataframe=None):
    res = []
    df_info["uknown_count"] = dataframe.eq("?").sum().values
    for idx, row in df_info.iterrows():
        column = row["column_name"]
        if row["column_type"] == "continuous":
            dataframe[column] = dataframe[column].astype(float)
            res.append({"column_name":column, "NIU_count":0})
        else:
            dataframe[column] = dataframe[column].astype(str)
            dataframe_NIU_mask = dataframe[column].str.contains("Not in universe")
            if dataframe_NIU_mask.any():
                dataframe_NIU_uval = dataframe[column][dataframe_NIU_mask].unique().item()
                NIU_count = dataframe[dataframe[column] == dataframe_NIU_uval].shape[0]
                res.append({"column_name":column, "NIU_count":NIU_count})
            else:
                res.append({"column_name":column, "NIU_count":0})
    
    # matching nominal column sanity check
    if (df_info[df_info["column_type"]=="nominal"]["column_name"].values == dataframe.select_dtypes(object).columns).all():
        df_info = pd.concat([df_info, pd.DataFrame(res)["NIU_count"]], axis=1)
        df_info = df_info.assign(row_count=dataframe.shape[0])
    return df_info

In [6]:
data_info = [
    "|   91 distinct values for attribute #0 (age) continuous",
    "|    9 distinct values for attribute #1 (class of worker) nominal",
    "|   52 distinct values for attribute #2 (detailed industry recode) nominal",
    "|   47 distinct values for attribute #3 (detailed occupation recode) nominal",
    "|   17 distinct values for attribute #4 (education) nominal",
    "| 1240 distinct values for attribute #5 (wage per hour) continuous",
    "|    3 distinct values for attribute #6 (enroll in edu inst last wk) nominal",
    "|    7 distinct values for attribute #7 (marital stat) nominal",
    "|   24 distinct values for attribute #8 (major industry code) nominal",
    "|   15 distinct values for attribute #9 (major occupation code) nominal",
    "|    5 distinct values for attribute #10 (race) nominal",
    "|   10 distinct values for attribute #11 (hispanic origin) nominal",
    "|    2 distinct values for attribute #12 (sex) nominal",
    "|    3 distinct values for attribute #13 (member of a labor union) nominal",
    "|    6 distinct values for attribute #14 (reason for unemployment) nominal",
    "|    8 distinct values for attribute #15 (full or part time employment stat) nominal",
    "|  132 distinct values for attribute #16 (capital gains) continuous",
    "|  113 distinct values for attribute #17 (capital losses) continuous",
    "| 1478 distinct values for attribute #18 (dividends from stocks) continuous",
    "|    6 distinct values for attribute #19 (tax filer stat) nominal",
    "|    6 distinct values for attribute #20 (region of previous residence) nominal",
    "|   51 distinct values for attribute #21 (state of previous residence) nominal",
    "|   38 distinct values for attribute #22 (detailed household and family stat) nominal",
    "|    8 distinct values for attribute #23 (detailed household summary in household) nominal",
    "|   10 distinct values for attribute #24 (migration code-change in msa) nominal",
    "|    9 distinct values for attribute #25 (migration code-change in reg) nominal",
    "|   10 distinct values for attribute #26 (migration code-move within reg) nominal",
    "|    3 distinct values for attribute #27 (live in this house 1 year ago) nominal",
    "|    4 distinct values for attribute #28 (migration prev res in sunbelt) nominal",
    "|    7 distinct values for attribute #29 (num persons worked for employer) continuous",
    "|    5 distinct values for attribute #30 (family members under 18) nominal",
    "|   43 distinct values for attribute #31 (country of birth father) nominal",
    "|   43 distinct values for attribute #32 (country of birth mother) nominal",
    "|   43 distinct values for attribute #33 (country of birth self) nominal",
    "|    5 distinct values for attribute #34 (citizenship) nominal",
    "|    3 distinct values for attribute #35 (own business or self employed) nominal",
    "|    3 distinct values for attribute #36 (fill inc questionnaire for veteran's admin) nominal",
    "|    3 distinct values for attribute #37 (veterans benefits) nominal",
    "|   53 distinct values for attribute #38 (weeks worked in year) continuous",
    "|    2 distinct values for attribute #39 (year) nominal",
]

In [7]:
s_data_info = pd.Series(data_info)\
    .str.replace("|", "")\
    .str.replace("distinct values for attribute #", ",")\
    .str.replace("(", ",")\
    .str.replace(")", ",")\
    .str.replace("'","")\
    .str.strip()
df_data_info = s_data_info.str.split(",", expand=True).drop(1,axis=1)
df_data_info.columns = ["nunique", "column_name", "column_type"]
df_data_info["nunique"] = df_data_info["nunique"].astype(int)
df_data_info.loc[40] = [2, "target", "nominal"]
df_data_info = df_data_info.map(lambda x: x.strip() if isinstance(x, str) else x)

In [8]:
df0 = pd.read_csv(
    filepath_or_buffer="data/census_income_learn.csv",
    header=None).drop(24,axis=1)

print(df0.shape)
print(df0.duplicated().sum()) # different total than metadata file (46627 vs.46716)
if (df0.nunique().reset_index(drop=True) == df_data_info["nunique"]).all():
    print("renaming columns")
    df0.columns = df_data_info["column_name"].tolist()
df0 = df0.map(lambda x: x.strip() if isinstance(x, str) else x)
df0["target"] = df0["target"].str.replace(".", "")

(199523, 41)
46627
renaming columns


In [9]:
# if edu is Children then target < 50k
df0[df0["education"]=="Children"]["target"].value_counts()
df1 = df0[df0["education"]!="Children"].reset_index(drop=True)
print(df1.shape)
print(df1.duplicated().sum())

(152101, 41)
11915


In [14]:
df_info_res = append_anomaly_counts(df_info=df_data_info, dataframe=df1)
df_info_res

,nunique,column_name,column_type,uknown_count,NIU_count,row_count
0,91,age,continuous,0,0,152101
1,9,class of worker,nominal,0,52823,152101
2,52,detailed industry recode,nominal,0,0,152101
3,47,detailed occupation recode,nominal,0,0,152101
4,17,education,nominal,0,0,152101
5,1240,wage per hour,continuous,0,0,152101
6,3,enroll in edu inst last wk,nominal,0,139522,152101
7,7,marital stat,nominal,0,0,152101
8,24,major industry code,nominal,0,53262,152101
9,15,major occupation code,nominal,0,53262,152101
